
# Dynamic annotating methods
This notebook demonstrates advanced ways of using OpenAI's function calling feature.
    

[!NOTE] : This notebook is a continuation of the [article](articles/dynamic_annotation_function_calling_and_agents.md) on dynamic annotation and function calling.


## Importing the required libaries

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
import os
from inspect import signature, Parameter
import functools
import re
from typing import Callable, Dict, List

## Loading Environment Variables

load_dotenv()

## Basic Function Calling Example

client = OpenAI()
client.key = os.getenv("OPENAI_API_KEY")
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather_forecast",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        }
    }
]
messages = [{"role": "user", "content": "What's the weather like in Boston today?"}]
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
    tool_choice="auto"
)

print(completion)

## Decorating functions


Keep in mind this class we will be using to annotate all other methods we want to expose to OpenAI models.


In [ ]:
def parse_docstring(func: Callable) -> Dict[str, str]:
    doc = func.__doc__
    if not doc:
        return {}
    param_re = re.compile(r':param\s+(\w+):\s*(.*)')
    param_descriptions = {}
    for line in doc.split('\n'):
        match = param_re.match(line.strip())
        if match:
            param_name, param_desc = match.groups()
            param_descriptions[param_name] = param_desc
    return param_descriptions

def function_schema(name: str, description: str, required_params: List[str]):
    def decorator_function(func: Callable) -> Callable:
        if not all(param in signature(func).parameters for param in required_params):
            raise ValueError(f"Missing required parameters in {func.__name__}")
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            return func(*args, **kwargs)
        params = signature(func).parameters
        param_descriptions = parse_docstring(func)
        serialized_params = {
            param_name: {
                "type": "string",
                "description": param_descriptions.get(param_name, "No description")
            }
            for param_name in required_params
        }
        wrapper.schema = {
            "name": name,
            "description": description,
            "parameters": {
                "type": "object",
                "properties": serialized_params,
                "required": required_params
            }
        }
        return wrapper
    return decorator_function

## Example of decorator

When writing the `description`, try to be as clear as possible, in my experience the more clear the better for the model to understand the method.

Tip: If your method has multiple or dynamic paramters, explain how the parameters are going to be used, rather than explain the function itself.

In [ ]:
@function_schema(
    name="get_weather_forecast",
    description="Finds information on the forecast for a specific location.",
    required_params=["location"]
)
def get_weather_forecast(location: str):
    return f"Forecast for {location} is ..."

## Function registry

We now have the decorator, we just need now to registry all the functions we want to expose to the model. This is done by using the `register` method of the `FunctionRegistry` class.

In [ ]:
import importlib.util
from pathlib import Path
import json
import logging

class FunctionsRegistry:
    def __init__(self) -> None:
        self.functions_dir = Path(__file__).parent.parent / 'functions'
        self.registry: Dict[str, callable] = {}
        self.schema_registry: Dict[str, Dict] = {}
        self.load_functions()

    def load_functions(self) -> None:
        if not self.functions_dir.exists():
            logging.error(f"Functions directory does not exist: {self.functions_dir}")
            return
        for file in self.functions_dir.glob('*.py'):
            module_name = file.stem
            if module_name.startswith('__'):
                continue
            spec = importlib.util.spec_from_file_location(module_name, file)
            if spec and spec.loader:
                module = importlib.util.module_from_spec(spec)
                spec.loader.exec_module(module)
                for attr_name in dir(module):
                    attr = getattr(module, attr_name)
                    if callable(attr) and hasattr(attr, 'schema'):
                        self.registry[attr_name] = attr
                        self.schema_registry[attr_name] = attr.schema

    def resolve_function(self, function_name: str, arguments_json: Optional[str] = None):
        func = self.registry.get(function_name)
        if not func:
            raise ValueError(f"Function {function_name} is not registered.")
        try:
            if arguments_json is not None:
                arguments_dict = json.loads(arguments_json) if isinstance(arguments_json, str) else arguments_json
                return func(**arguments_dict)
            else:
                return func()
        except json.JSONDecodeError:
            logging.error("Invalid JSON format.")
            return None
        except Exception as e:
            logging.error(f"Error when calling function {function_name}: {e}")
            return None

    def mapped_functions(self) -> List[Dict]:
        return [
            {
                "type": "function",
                "function": func_schema
            }
            for func_schema in self.schema_registry.values()
        ]

    def generate_schema_file(self) -> None:
        schema_path = self.functions_dir / 'function_schemas.json'
        with schema_path.open('w') as f:
            json.dump(list(self.schema_registry.values()), f, indent=2)

    def get_registry_contents(self) -> List[str]:
        return list(self.registry.keys())

    def get_schema_registry(self) -> List[Dict]:
        return list(self.schema_registry.values())

## Example

def main() -> None:
    try:
        client = OpenAI()
        client.key = os.getenv("OPENAI_API_KEY")
        if not client.key:
            raise ValueError("API key not found in environment variables.")
        tools = FunctionsRegistry()
        messages = [{"role": "user", "content": "what's the weather forecast for Wellington, New Zealand"}]
        completion = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
            tools=tools.mapped_functions(),
            tool_choice="auto"
        )
        print(completion)
    except Exception as e:
        logging.error(f"An error occurred: {e}")

if __name__ == "__main__":
    main()